# GAMA-12 master catalogue: Flags

In [1]:
import numpy as np
from astropy.table import Table

import itertools

from herschelhelp_internal.flagging import flag_outliers

In [2]:
SUFFIX = "20180218"
FIELD = "GAMA-12"
catname = "../../dmu1/dmu1_ml_GAMA-12/data/master_catalogue_gama-12_{}.fits".format(SUFFIX)
master_catalogue = Table.read(catname)

In [3]:
u_bands = [           "KIDS u"]
g_bands = ["DECam g", "KIDS g", "SUPRIME g", "GPC1 g"]
r_bands = ["DECam r", "KIDS r", "SUPRIME r", "GPC1 r"]
i_bands = [           "KIDS i", "SUPRIME i", "GPC1 i"]
z_bands = ["DECam z",           "SUPRIME z", "GPC1 z", "VIRCAM Z"]
y_bands = [                     "SUPRIME y", "GPC1 y", "VIRCAM Y", "WFCAM Y"]
J_bands = [                                            "VIRCAM J", "WFCAM J"]
H_bands = [                                            "VIRCAM H", "WFCAM H"]
K_bands = [                                            "VIRCAM K", "WFCAM K"]

all_bands = [u_bands, g_bands, r_bands, i_bands, z_bands, y_bands, J_bands, H_bands, K_bands]

## 1. Magnitudes and magnitude erros

In [4]:
def flag_mag(flagcol, mask):
    
    # Add flag columns if does not exist
    if flagcol not in master_catalogue.colnames:
        master_catalogue[flagcol] = np.zeros(len(master_catalogue), dtype=bool)
    
    # Flagged
    master_catalogue[flagcol][mask] = np.ones(len(mask), dtype=bool)
    print('    Number of flagged objects:', len(master_catalogue[flagcol][mask]))

### 1.a Pan-STARRS Aperture and Total magnitude errors

In [5]:
## dmu0: Pan-STARRS forced photometry cat 
gpc1_err = 0.0010860000038519502
bands = ["GPC1 g", "GPC1 r", "GPC1 i", "GPC1 z", "GPC1 y"]

for i, band in enumerate(bands):
    print(band)
    basecol = band.replace(" ", "_").lower()
    
    ecol_ap, ecol_tot = "merr_ap_{}".format(basecol), "merr_{}".format(basecol)
    flagcol_ap, flagcol_tot = "flag_{}_ap".format(basecol), "flag_{}".format(basecol)
    
    mask_ap  = np.where(master_catalogue[ecol_ap]  == gpc1_err)[0]
    mask_tot = np.where(master_catalogue[ecol_tot] == gpc1_err)[0]
    
    print('  Aperture magnitude')
    flag_mag(flagcol_ap, mask_ap)
    print('  Total magnitude')
    flag_mag(flagcol_tot, mask_tot)
    

GPC1 g
  Aperture magnitude
    Number of flagged objects: 555
  Total magnitude
    Number of flagged objects: 611
GPC1 r
  Aperture magnitude
    Number of flagged objects: 1042
  Total magnitude
    Number of flagged objects: 1038
GPC1 i
  Aperture magnitude
    Number of flagged objects: 980
  Total magnitude
    Number of flagged objects: 1036
GPC1 z
  Aperture magnitude
    Number of flagged objects: 781
  Total magnitude
    Number of flagged objects: 810
GPC1 y
  Aperture magnitude
    Number of flagged objects: 300
  Total magnitude
    Number of flagged objects: 295


### 2.a DECaLS Total magnitudes

In [6]:
decam_mag = 14.999935
bands = ["DECam g", "DECam r", "DECam z"]

for i, band in enumerate(bands):
    print(band)
    basecol = band.replace(" ", "_").lower()
    
    col = "m_{}".format(basecol)
    flagcol = "flag_{}".format(basecol)
    
    mask  = np.where((master_catalogue[col]  == decam_mag) | (master_catalogue[col] < 7))[0]
    
    print('  Total magnitude')
    flag_mag(flagcol, mask)

DECam g


/opt/anaconda3/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in less
  return getattr(self.data, op)(other)


  Total magnitude
    Number of flagged objects: 126
DECam r
  Total magnitude
    Number of flagged objects: 115
DECam z
  Total magnitude
    Number of flagged objects: 1


## 2. Outliers

In [7]:
for band_of_a_kind in all_bands:
    for band1, band2 in itertools.combinations(band_of_a_kind, 2):
        #print(band1, band2)

        basecol1, basecol2 = band1.replace(" ", "_").lower(), band2.replace(" ", "_").lower()
        
        # Aperture mag
        col1, col2 = "m_ap_{}".format(basecol1), "m_ap_{}".format(basecol2)
        ecol1, ecol2 = "merr_ap_{}".format(basecol1), "merr_ap_{}".format(basecol2)
        flagcol1, flagcol2 = "flag_ap_{}".format(basecol1), "flag_ap_{}".format(basecol2)
        
        try:
            master_catalogue = flag_outliers(master_catalogue, col1, col2,
                      ecol1, ecol2,
                      flagcol1, flagcol2,
                      labels=("{} (aperture)".format(band1), "{} (aperture)".format(band2)))
        except KeyError:
            print("One of {} and {} is not present in the catalogue.".format(col1, col2))
                      
        
        # Tot mag
        col1, col2 = "m_{}".format(basecol1), "m_{}".format(basecol2)              
        ecol1, ecol2 = "merr_{}".format(basecol1), "merr_{}".format(basecol2)              
        flagcol1, flagcol2 = "flag_{}".format(basecol1), "flag_{}".format(basecol2)
              
        try:
            master_catalogue = flag_outliers(master_catalogue, col1, col2, 
                      ecol1, ecol2,
                      flagcol1, flagcol2,
                      labels=("{} (total)".format(band1), "{} (total)".format(band2)))   
        except KeyError:
            print("One of {} and {} is not present in the catalogue.".format(col1, col2))         
                      

One of m_ap_decam_g and m_ap_kids_g is not present in the catalogue.
One of m_decam_g and m_kids_g is not present in the catalogue.
SUPRIME g (aperture) - DECam g (aperture):
  Number of outliers: 4
No sources have both DECam g (total) and SUPRIME g (total) values.

GPC1 g (aperture) - DECam g (aperture):
  Number of outliers: 1009
GPC1 g (total) - DECam g (total):
  Number of outliers: 30
One of m_ap_kids_g and m_ap_suprime_g is not present in the catalogue.
One of m_kids_g and m_suprime_g is not present in the catalogue.
One of m_ap_kids_g and m_ap_gpc1_g is not present in the catalogue.
One of m_kids_g and m_gpc1_g is not present in the catalogue.
GPC1 g (aperture) - SUPRIME g (aperture):
  Number of outliers: 0
No sources have both SUPRIME g (total) and GPC1 g (total) values.

One of m_ap_decam_r and m_ap_kids_r is not present in the catalogue.
One of m_decam_r and m_kids_r is not present in the catalogue.
SUPRIME r (aperture) - DECam r (aperture):
  Number of outliers: 0
SUPRIME r

## 3. Save table

In [8]:
flag_cols = ["help_id"]
for col in master_catalogue.colnames:
    if col.startswith("flag_"):
        flag_cols += [col]
new_catname = "./data/{}_{}_flags.fits".format(FIELD.lower(),SUFFIX)
master_catalogue[flag_cols].write(new_catname, overwrite = True)